In [3]:
#!sudo pip install --upgrade transformers

In [1]:
from refractml import *
from refractml.constants import MLModelFlavours
# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
# import json
# print(json.dumps(payload,indent=2))

In [4]:
context ={
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}

In [6]:
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        self.model_loaded = None
        self.model_name ="/data/huggingface/cache/models--openchat--openchat_3.5/snapshots/2d39b2e68fc16446e1760700e0de14721434cbbf"
        #self.date_input = {
        #    "start_date": "01/01/2020",
        #   "end_date": "15/09/2023"}

        self.base_prompt = "" #Add your prompt here 
        from transformers import AutoTokenizer, AutoModelForCausalLM
        import time
        import torch
        from huggingface_hub.utils import disable_progress_bars
        disable_progress_bars()
        if self.model_loaded is None:
            print("Model is Loading from data cache")
            self.model_loaded = AutoModelForCausalLM.from_pretrained(self.model_name, device_map="auto", torch_dtype=torch.bfloat16)
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
            #self.tokenizer.pad_token = self.tokenizer.eos_token
            #self.tokenizer.padding_side = "right"
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[{"question": "","context":""},{"question": "","context":""},...]

        :return: (n_inputs, payload's)

        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (len(raw_payload), raw_payload) 
    
    def pre_processing_fn(self,payload):
        return payload

    
    def prediction_fn(self,
                      model: Any,
                      input_query
                       ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        question = input_query.get("question")
        context = input_query.get("context")
        date_info = input_query.get("date_info")
        query = """Now Based on this following context: \n """+ str(context) + "\n return a response for this question:\n" + question + "\n"
        prompt = " "#Reneder your inputs in the prompt form here
        print("Prompting has started")
        import time
        start = time.time()
        model_input = self.tokenizer(prompt, return_tensors="pt")#.to("cuda")
        response=  self.tokenizer.decode(self.model_loaded.generate(**model_input, max_length=1024*6)[0], skip_special_tokens=True)
        print(f"Total time taken = {time.time()-start}")
        #response = response[0]["generated_text"]
        preds = response.split("[SOL]")[-1].split("[/SOL]")[0]
        print("prediction is \n: ",preds)
        return preds

In [1]:
# check the model class
score = ScoreTemplateExample()
req = requests.Request()
#payload that is required for inference
req.json = {"payload":[{}]}
score.score(None, req, dry_run=True)

In [8]:
model="/data/huggingface/cache/models--openchat--openchat_3.5/snapshots/2d39b2e68fc16446e1760700e0de14721434cbbf"
#model=score_.model_loaded

In [9]:
register_model(model,
               ScoreTemplateExample,
               "Open_Chat_iMQL_Generator_for_GPU_01",
               "IMQL_Generator_using_open_chat_3_5_for_GPU01",
               MLModelFlavours.pytorch,
               init_script="""sudo chmod -R 777 /home/mosaic-ai/.cache\\npip install torch \\npip install --upgrade transformers\\npip install accelerate"""
              )

INFO:root:PASS:: Mandatory Validation :: dict_keys(['name', 'description', 'flavour', 'scoring_func'])
INFO:root:PASS:: AlphaNumeric Validation :: dict_keys(['name'])
INFO:root:PASS:: Validation/IfPresentTypeCheck :: dict_keys(['schema', 'metadata_info', 'tags'])
INFO:root:PASS:: Validation/IfPresentSubfieldMustExist :: dict_keys(['kyd'])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "POST /registry/api/v1/ml-model/register HTTP/1.1" 200 6940
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/6ed23fdd-6fff-4354-9047-913b00cf916a HTTP/1.1" 200 6933
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/6ed23fdd-6fff-4354-9047-913b00cf916a HTTP/1.1"